### **QUERIES**

Here, the queries are launched.
First, define the class connection.

In [2]:
from neo4j import GraphDatabase
import pandas as pd

In [6]:
# Connection settings
uri = "neo4j://localhost:7687"
pass = "pass"
driver = GraphDatabase.driver(uri, auth=("neo4j", pass))

QUERY 1. Find the top 3 most cited papers of each conference.

In [7]:
def query1(tx):
    answer = tx.run("""

    MATCH (y:Article)<-[:cited_by]-(x:Article)-[:published_in]->(z:Proceeding)-[:belongs_to]->(w:Conference)
    WITH w.name AS ConferenceTitle, z.year AS Year, x.article_id AS ArticleTitle, count(*) AS Citations
    ORDER BY ConferenceTitle, Year, Citations DESC
    WITH ConferenceTitle, Year, collect({ArticleTitle:ArticleTitle, Citations:Citations}) AS ArticlesCited, collect(Citations)[2] AS ThirdPlaceCitations
    WITH ConferenceTitle, Year, ArticlesCited, ThirdPlaceCitations, reduce(draws=0, i in range(2,size(ArticlesCited)-1) | CASE WHEN ArticlesCited[i].Citations = ThirdPlaceCitations THEN draws+1 ELSE draws END) AS ThirdPlaces
    WITH ConferenceTitle, Year, ThirdPlaces, ArticlesCited[0..(2+ThirdPlaces)] AS ArticlesCited
    UNWIND ArticlesCited AS TopCitedArticles
    RETURN ConferenceTitle, Year, TopCitedArticles.ArticleTitle AS TopCitedArticle, TopCitedArticles.Citations AS Citations

    """)
    
    articles = []
    for article in answer:
        articles.append(article)
    return articles

In [8]:
with driver.session() as session:
    articles = session.read_transaction(query1)

In [9]:
# Create table with answer
cols=["conference","year","article", "citations"]
articles_df = pd.DataFrame(columns=cols)
for i in range(len(articles)):
    data = [
        articles[i].get("ConferenceTitle"),
        articles[i].get("Year"),
        articles[i].get("TopCitedArticle"),
        articles[i].get("Citations")
        ]
    new_df = pd.DataFrame([data], columns=cols)
    articles_df = pd.concat([articles_df, new_df])
articles_df

,conference,year,article,citations
0,IEEE International Conference on Software Mai...,2006,2618931,26
0,IEEE International Conference on Software Mai...,2006,1436545,15
0,IEEE International Conference on Software Mai...,2006,63488,15
0,IEEE International Conference on Software Mai...,2008,583151,19
0,IEEE International Conference on Software Mai...,2008,2053033,17
...,...,...,...,...
0,"User Interfaces , Australasian User Interface...",2009,2825118,18
0,"User Interfaces , Australasian User Interface...",2009,2285044,17
0,Visual Proceedings of e Annual Conference on ...,2021,558299,13
0,Visual Proceedings of e Annual Conference on ...,2021,2395313,12


QUERY 2. For each conference find its community: i.e., those authors that have published papers on that conference in, at least, 4 different editions.

In [10]:
def query2(tx):
    answer = tx.run("""

    MATCH (a:Author)<-[:written_by]-(b:Article)-[:published_in]->(c:Proceeding)-[:belongs_to]->(d:Conference)
    WITH DISTINCT d.name AS ConferenceTitle, c.year AS Edition, a.author_id AS AuthorId, a.name AS AuthorName
    WITH ConferenceTitle, AuthorId, AuthorName, count(*) AS Publications
    WITH ConferenceTitle, collect({AuthorId:AuthorId, AuthorName:AuthorName, Publications:Publications}) AS AuthorsPublications
    UNWIND [t1 IN AuthorsPublications WHERE t1.Publications >= 2] AS TopAuthors
    RETURN ConferenceTitle, TopAuthors.AuthorId AS AuthorId, TopAuthors.AuthorName AS AuthorName, TopAuthors.Publications AS Publications

    """)
    
    authors = []
    for author in answer:
        authors.append(author)
    return authors

In [11]:
with driver.session() as session:
    authors = session.read_transaction(query2)

In [12]:
# Create table with answer
cols=["conference","author id","author name", "publications"]
authors_df = pd.DataFrame(columns=cols)
for i in range(len(authors)):
    data = [
        authors[i].get("ConferenceTitle"),
        authors[i].get("AuthorId"),
        authors[i].get("AuthorName"),
        authors[i].get("Publications")
        ]
    new_df = pd.DataFrame([data], columns=cols)
    authors_df = pd.concat([authors_df, new_df])
authors_df

,conference,author id,author name,publications
0,Proceedings of e ACM Cloud Computing Security...,9016863,Han-Chieh Chao,4
0,Proceedings of e Biannual Conference of e Ita...,9016863,Han-Chieh Chao,2


QUERY 3: Find the impact factors of the journals in your graph

In [13]:
def query3(tx):
    answer = tx.run("""

    MATCH (article:Article)-[:published_in]->(volume:Volume)-[:belongs_to]->(journal:Journal)
    WITH journal AS Journal, toInteger(date(volume.date).year) AS Year
    MATCH (article:Article)-[:published_in]->(volume:Volume)-[:belongs_to]->(Journal)
    WHERE toInteger(date(volume.date).year) = Year-1 OR toInteger(date(volume.date).year) = Year-2
    WITH Journal, Year, COUNT(article) AS Articles_2years
    MATCH (volume_proceeding)<-[:published_in]-(citing_article:Article)<-[:cited_by]-(article:Article)-[:published_in]->(volume:Volume)-[:belongs_to]->(Journal)
    WHERE (toInteger(date(volume_proceeding.date).year) = Year OR volume_proceeding.year = Year) AND (toInteger(date(volume.date).year) = Year-1 OR toInteger(date(volume.date).year) = Year-2)
    WITH Journal, Year, Articles_2years, count(*) AS Citations_2years
    WHERE Articles_2years > 0
    RETURN Journal, Year, Articles_2years, Citations_2years, toFloat(Citations_2years) / Articles_2years  AS ImpactFactor

    """)
    
    journals = []
    for journal in answer:
        journals.append(journal)
    return journals

In [14]:
with driver.session() as session:
    journals = session.read_transaction(query3)

In [15]:
# Create table with answer
cols=["journal","year","articles last 2 years", "citations last 2 years", "impact factor"]
journals_df = pd.DataFrame(columns=cols)
for i in range(len(journals)):
    data = [
        journals[i].get("Journal").get("name"),
        journals[i].get("Year"),
        journals[i].get("Articles_2years"),
        journals[i].get("Citations_2years"),
        journals[i].get("ImpactFactor")
        ]
    new_df = pd.DataFrame([data], columns=cols)
    journals_df = pd.concat([journals_df, new_df])
journals_df

,journal,year,articles last 2 years,citations last 2 years,impact factor
0,J. Econ. Theory,2021,3,8,2.666667
0,J. Econ. Theory,2018,3,3,1.0
0,J. Econ. Theory,2020,4,3,0.75
0,J. Econ. Theory,2019,3,7,2.333333
0,J. Econ. Theory,2017,4,2,0.5
...,...,...,...,...,...
0,IEEE Commun. Lett.,2019,5,7,1.4
0,VLDB J.,2020,1,2,2.0
0,IEEE ACM Trans. Audio Speech Lang. Process.,2021,2,1,0.5
0,Softw. Test. Verification Reliab.,2017,1,1,1.0


QUERY 4. Find the h-indexes of the authors in your graph

In [16]:
def query4(tx):
    answer = tx.run("""

    MATCH (auth:Author)<-[w:written_by]-(:Article)
    WITH auth, count(w) AS numberOfArt
    MATCH (auth)<-[:written_by]-(art:Article)
    OPTIONAL MATCH (art)<-[cited_by]-(cityingArt:Article)
    WITH auth, numberOfArt, art, count(cityingArt) as numberOfCitations
    ORDER BY auth, numberOfCitations DESC
    WITH auth, numberOfArt, collect(numberOfCitations) AS citations
    RETURN auth, reduce(hfact=0, i in (range(0,numberOfArt-1)) | CASE WHEN i<citations[i] THEN hfact+1 ELSE hfact END ) AS Hfactor
    ORDER BY Hfactor DESC

    """)
    
    authors = []
    for author in answer:
        authors.append(author)
    return authors

In [17]:
with driver.session() as session:
    authors = session.read_transaction(query4)

In [18]:
# Create table with answer
cols=["author id","author name", "hfactor"]
authors_df = pd.DataFrame(columns=cols)
for i in range(len(authors)):
    data = [
        authors[i].get("auth").get("author_id"),
        authors[i].get("auth").get("name"),
        authors[i].get("Hfactor")
        ]
    new_df = pd.DataFrame([data], columns=cols)
    authors_df = pd.concat([authors_df, new_df])
authors_df

,author id,author name,hfactor
0,8973836,Victor C. M. Leung,4
0,8973885,Yuan Xie 0001,4
0,8991837,Wei Wang,4
0,9057663,Chin-Chen Chang 0001,4
0,9221398,Vince D. Calhoun,4
...,...,...,...
0,9144543,Sayantan Das,1
0,9144576,Lily L. Liu,1
0,9144633,Kenneth Zeger,1
0,9144651,Sophie Spirkl,1
